In [37]:
import numpy as np
import pandas as pd
import json

In [38]:
data=pd.read_json("/content/Books_small.json",lines=True)
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1E5ZR1Z4OQJG,1495329321,"Pure Jonel ""Pure Jonel""","[0, 0]",Da Silva takes the divine by storm with this u...,4,An amazing first novel,1396137600,"03 30, 2014"
1,A30PZPI6FPH0A7,0399157565,Jackmollie,"[0, 0]",For me personally it's the most disappointing ...,2,disappointed,1400112000,"05 15, 2014"
2,A1GQ2UI5BKCCRD,0984528105,Gail Hodges,"[0, 0]","Very simple book, but leaves you feeling good....",4,Good book,1401235200,"05 28, 2014"
3,A2DF4LQQI6KSQ2,0804139024,Olga,"[0, 0]",I read a library copy of this exceptionally we...,5,Science Fiction at its best!,1396483200,"04 3, 2014"
4,A1UAMAWY966P2,0765317583,Nadyne M Ichimura,"[0, 0]",With the government knowing this could happen ...,5,Excellent story,1397001600,"04 9, 2014"


We only need the reviewtext and the score for classifying the review. Storing both of them in a list.

In [39]:
x=list(data.iloc[:,4:6].values)
print(x[4][1])

5


In [40]:
class Review:
  def __init__(self, text, score):
    self.text=text
    self.score=score
    self.sentiment = self.get_Sentiment()

  def get_Sentiment(self):
    if self.score<=2:
      return "NEGATIVE"
    elif self.score==3:
      return "NEUTRAL"
    else:
      return "POSITIVE"




We can make a class for getting score and everything, to make it easier and cleaner.

In [41]:
fname="/content/Books_small.json"
reviews=[]
with open(fname) as f:
  for line in f:
    #print(line)
    #break (to check if it works or not)
    review=json.loads(line)
    #print(review["reviewText"]) (to check if it works or not))
    #print(review["overall"])    (to check if it works or not))
    #break
    reviews.append(Review(review["reviewText"],review["overall"]))

print(reviews[5].text)

Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!


Prepping Data:

In [42]:
len(reviews)

1000

In [43]:
from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(reviews,test_size=0.33,random_state=0)

In [44]:
len(x_train)

670

In [45]:
print(x_train[0].score)

5.0


In [46]:
print(x_train[1].sentiment)

POSITIVE


In [50]:
train_t=[x.text for x in x_train]
train_s=[x.sentiment for x in x_train]
test_t=[x.text for x in x_test]
test_s=[x.sentiment for x in x_test]
train_t[0]
train_s[0]

'POSITIVE'

In [51]:
test_s[1]

'POSITIVE'

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_t_vectors=vectorizer.fit_transform(train_t)
train_s_vectors=vectorizer.transform(train_s)
test_t_vectors=vectorizer.transform(test_t)
test_s_vectors=vectorizer.transform(test_s)

In [57]:
print(train_t_vectors[0].toarray())

[[0 0 0 ... 0 0 0]]


In [63]:
#SVM
from sklearn import svm
c_svc=svm.SVC(kernel="linear")
c_svc.fit(train_t_vectors,train_s)

SVC(kernel='linear')

In [64]:
test_s[0]

'POSITIVE'

In [71]:
print(test_s_vectors[0])

  (0, 5113)	1


In [73]:
c_svc.predict(test_t_vectors[0])

array(['NEUTRAL'], dtype='<U8')

In [75]:
from sklearn.tree import DecisionTreeClassifier
c_t=DecisionTreeClassifier()
c_t.fit(train_t_vectors,train_s)

DecisionTreeClassifier()

In [76]:
c_t.predict(test_t_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [77]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(train_t_vectors,train_s)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [78]:
LR.predict(test_t_vectors[0])

array(['NEUTRAL'], dtype='<U8')

In [80]:
LR.score(test_t_vectors,test_s)

0.8212121212121212

In [81]:
c_t.score(test_t_vectors,test_s)

0.7606060606060606

In [82]:
c_svc.score(test_t_vectors,test_s)

0.803030303030303

In [84]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(test_s,LR.predict(test_t_vectors))
print(cf)

[[  1   2  18]
 [  0   5  31]
 [  2   6 265]]
